In [ ]:
# DIT SCRIPT IS ECHT BIZAR KAKA 
# NIET GEBRUIKEN



import pandas as pd
from datetime import timedelta, datetime
from collections import defaultdict
import random
import math

def generate_initial_solution(dienstregeling, afstandsmatrix):
    """
    Generate an initial feasible solution for the bus planning problem.
    """
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Add a new 'omloop nummer' column and initialize it
    planning_data['omloop nummer'] = range(1, len(planning_data) + 1)

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    for _, row in planning_data.iterrows():
        # Implement the original logic from generate_feasible_plan here
        # ...
        pass

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    initial_solution = pd.DataFrame(bus_schedules)
    return initial_solution
def generate_initial_solution(dienstregeling, afstandsmatrix):
    """
    Generate an initial feasible solution for the bus planning problem.
    """
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Debugging: Check the merged planning data
    print("Merged planning data:")
    print(planning_data.head())  # Print the first few rows to verify the merge

    # Add a new 'omloop nummer' column and initialize it
    planning_data['omloop nummer'] = range(1, len(planning_data) + 1)

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Debugging: Check if the sorting and omloop nummer assignment worked
    print("Sorted and processed planning data:")
    print(planning_data.head())  # Print the first few rows to verify sorting

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    # Check if the planning_data is empty or has no trips
    if planning_data.empty:
        print("Warning: No trips found in the planning data!")
        return pd.DataFrame()  # Return an empty DataFrame if no data is found

    for _, row in planning_data.iterrows():
        # Implement the original logic from generate_feasible_plan here
        # For now, just add a placeholder trip
        bus_schedules.append({
            'buslijn': row['buslijn'],
            'startlocatie': row['startlocatie'],
            'eindlocatie': row['eindlocatie'],
            'vertrektijd': row['vertrektijd'],
            'omloop nummer': row['omloop nummer']
        })

    # Convert the bus_schedules list to a DataFrame for output
    initial_solution = pd.DataFrame(bus_schedules)

    # Debugging: Check the generated initial solution
    print("Generated initial solution:")
    print(initial_solution.head())  # Print the first few rows to verify

    return initial_solution

def calculate_energy_consumption(distance):
    """
    Calculate the energy consumption for a bus trip based on the distance.
    """
    return distance * 2.5 / 1000  # kWh per meter

def calculate_solution_cost(solution):
    """
    Calculate the cost of a given solution, which is the number of unique omloop nummers.
    """
    if isinstance(solution, pd.DataFrame):
        if 'omloop nummer' in solution.columns:
            return len(set(solution['omloop nummer']))
        else:
            return 0
    else:
        return len(set([trip['omloop nummer'] for trip in solution]))

def simulate_annealing(dienstregeling, afstandsmatrix, initial_temperature, cooling_rate, max_iterations):
    """
    Perform simulated annealing optimization to find a near-optimal solution for the bus planning problem.
    """
    # Generate an initial feasible solution
    current_solution = generate_initial_solution(dienstregeling, afstandsmatrix)
    try:
        current_cost = calculate_solution_cost(current_solution)
    except KeyError as e:
        print(f"KeyError occurred: {e}")
        print("Current solution DataFrame:")
        print(current_solution)
        raise

    best_solution = current_solution
    best_cost = current_cost

    temperature = initial_temperature

    for _ in range(max_iterations):
        # Generate a neighbor solution
        neighbor_solution = generate_neighbor_solution(current_solution)
        neighbor_cost = calculate_solution_cost(neighbor_solution)

        # Calculate the cost difference
        cost_difference = neighbor_cost - current_cost

        # Accept the neighbor solution with a certain probability
        if cost_difference < 0 or random.uniform(0, 1) < math.exp(-cost_difference / temperature):
            current_solution = neighbor_solution
            current_cost = neighbor_cost

            # Update the best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Decrease the temperature
        temperature *= cooling_rate

    return best_solution

def generate_neighbor_solution(current_solution):
    """
    Generate a neighbor solution by swapping the assignment of two trips to different buses.
    """
    # Create a copy of the current solution to modify
    neighbor_solution = current_solution.copy()

    # Check if there are enough trips to sample
    if len(neighbor_solution) < 2:
        # If there are fewer than 2 trips, just return the current solution (no changes)
        return neighbor_solution

    # Randomly select two trips to swap
    trips = neighbor_solution.sample(2)

    # Swap the 'omloop nummer' values for the two trips
    trips['omloop nummer'] = trips['omloop nummer'].iloc[::-1]

    # Replace the original trips with the swapped trips
    for index, row in trips.iterrows():
        neighbor_solution.loc[index, 'omloop nummer'] = row['omloop nummer']

    return neighbor_solution

def save_plan_to_excel(schedule_df, output_file):
    """
    Save the consolidated bus planning schedule to an Excel file on a single sheet.

    Parameters:
    schedule_df (pandas.DataFrame): DataFrame containing the feasible schedule.
    output_file (str): Path to the Excel file to save the schedule.
    """
    # Save the single-sheet DataFrame to an Excel file
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

initial_temperature = 1000
cooling_rate = 0.95
max_iterations = 10000

optimized_solution = simulate_annealing(dienstregeling, afstandsmatrix, initial_temperature, cooling_rate, max_iterations)

# Save the optimized solution to an Excel file
save_plan_to_excel(optimized_solution, 'optimized_plan_consolidated.xlsx')


Merged planning data:
  startlocatie vertrektijd eindlocatie  buslijn  min reistijd in min  \
0       ehvapt       06:04      ehvbst      401                   22   
1       ehvapt       06:34      ehvbst      401                   22   
2       ehvapt       06:56      ehvbst      401                   22   
3       ehvapt       07:11      ehvbst      401                   22   
4       ehvapt       07:26      ehvbst      401                   22   

   max reistijd in min  afstand in meters  
0                   25               9050  
1                   25               9050  
2                   25               9050  
3                   25               9050  
4                   25               9050  
Sorted and processed planning data:
    startlocatie vertrektijd eindlocatie  buslijn  min reistijd in min  \
85        ehvapt       00:01      ehvbst      401                   22   
251       ehvbst       00:09      ehvapt      401                   22   
86        ehvapt       

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import timedelta
import random

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def calculate_total_cost(schedule_df):
    """Calculate the total cost of a schedule, e.g., based on energy consumption."""
    # Cost could be based on total energy consumption and other criteria
    total_cost = schedule_df['energieverbruik'].abs().sum()  # Total energy cost as a starting example
    return total_cost

def generate_initial_solution(dienstregeling, afstandsmatrix):
    """Generate the initial feasible solution using the existing function."""
    return generate_feasible_plan(dienstregeling, afstandsmatrix)

def modify_schedule(schedule_df):
    """Make a small random modification to the schedule for Simulated Annealing."""
    # Randomly select two trips to swap or modify one trip’s omloop nummer
    modified_schedule = schedule_df.copy()
    
    # Select two random indices and swap their omloop nummers
    idx1, idx2 = random.sample(range(len(modified_schedule)), 2)
    modified_schedule.at[idx1, 'omloop nummer'], modified_schedule.at[idx2, 'omloop nummer'] = (
        modified_schedule.at[idx2, 'omloop nummer'], modified_schedule.at[idx1, 'omloop nummer']
    )
    
    # Recalculate the energy consumption if needed, or validate the SOC if necessary
    # This is just one way to modify; other constraints could be enforced here

    return modified_schedule

def simulated_annealing(dienstregeling, afstandsmatrix, initial_temp=1000, cooling_rate=0.003, max_iter=1000):
    # Step 1: Generate an initial feasible solution
    current_solution = generate_initial_solution(dienstregeling, afstandsmatrix)
    current_cost = calculate_total_cost(current_solution)
    
    # Keep track of the best solution
    best_solution = current_solution
    best_cost = current_cost
    
    temperature = initial_temp

    for i in range(max_iter):
        # Step 2: Generate a neighboring solution
        new_solution = modify_schedule(current_solution)
        new_cost = calculate_total_cost(new_solution)
        
        # Step 3: Calculate the cost difference
        cost_diff = new_cost - current_cost
        
        # Step 4: Decide whether to accept the new solution
        if cost_diff < 0 or np.exp(-cost_diff / temperature) > random.random():
            # Accept the new solution if it's better or by probability if worse
            current_solution = new_solution
            current_cost = new_cost
            
            # Update the best solution if the new solution is better
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost

        # Step 5: Cool down the temperature
        temperature *= (1 - cooling_rate)

        # Optional: Early stopping if temperature is very low or solution stabilizes
        if temperature < 1e-3:
            break

    return best_solution

def save_plan_to_excel(schedule_df, output_file):
    """Save the consolidated bus planning schedule to an Excel file on a single sheet."""
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'optimized_feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Run the Simulated Annealing optimization
optimized_plan_df = simulated_annealing(dienstregeling, afstandsmatrix)

# Save the optimized plan to an Excel file
save_plan_to_excel(optimized_plan_df, output_file)


NameError: name 'generate_feasible_plan' is not defined

In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import timedelta
import random
import pandas as pd
from collections import defaultdict
from datetime import timedelta

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def generate_feasible_plan(dienstregeling, afstandsmatrix):
    # Standardize location names to lowercase for consistency
    dienstregeling['startlocatie'] = dienstregeling['startlocatie'].str.lower()
    dienstregeling['eindlocatie'] = dienstregeling['eindlocatie'].str.lower()
    afstandsmatrix['startlocatie'] = afstandsmatrix['startlocatie'].str.lower()
    afstandsmatrix['eindlocatie'] = afstandsmatrix['eindlocatie'].str.lower()

    # Merge the dienstregeling with the afstandsmatrix to get the distance for each trip
    planning_data = pd.merge(
        dienstregeling, 
        afstandsmatrix, 
        on=['startlocatie', 'eindlocatie', 'buslijn'], 
        how='left'
    )

    # Sort the planning data by departure time
    planning_data = planning_data.sort_values('vertrektijd')

    # Initialize the bus schedules, state of charge (SOC), and charging schedules for each bus
    bus_schedules = []
    bus_soc = defaultdict(lambda: 300)  # Assuming 300 kWh battery capacity for each bus
    bus_charging = defaultdict(list)
    bus_last_location = defaultdict(lambda: 'ehvgar')  # Assume buses start at the garage
    bus_next_omloop = 1  # Start at omloop nummer 1

    # Define the date for all trips in the schedule
    schedule_date = "2024-08-29"

    for _, row in planning_data.iterrows():
        start_location = row['startlocatie']
        end_location = row['eindlocatie']
        distance = row['afstand in meters']  # in meters
        bus_line = row['buslijn']
        departure_time = pd.to_datetime(f"{schedule_date} {row['vertrektijd']}")

        # Calculate energy consumption for the trip
        energy_consumption = calculate_energy_consumption(distance)

        # Calculate arrival time using the minimum travel time in minutes
        travel_time = timedelta(minutes=int(row['min reistijd in min']))  # Convert to int
        arrival_time = departure_time + travel_time

        # Find an available bus or assign a new one
        available_bus = None
        for bus, soc in bus_soc.items():
            # Check if the bus is available, has sufficient SOC, and is not currently charging
            last_trip = [trip for trip in bus_schedules if trip['omloop nummer'] == bus]
            if last_trip and last_trip[-1]['eindtijd'] <= departure_time.time() and soc >= energy_consumption and not any(charging['starttijd'] <= departure_time.time() <= charging['eindtijd'] for charging in bus_charging[bus]):
                available_bus = bus
                break
        if available_bus is None:
            # If no bus is available, assign a new bus
            available_bus = bus_next_omloop
            bus_next_omloop += 1

        # Record the trip in the schedule with formatted start and end date-times
        bus_schedules.append({
            'startlocatie': start_location,
            'eindlocatie': end_location,
            'starttijd': departure_time.time(),
            'eindtijd': arrival_time.time(),
            'activiteit': 'dienst rit' if bus_line else 'materiaal rit',
            'buslijn': bus_line if bus_line else 'n/a',
            'energieverbruik': -energy_consumption if end_location == 'ehvgar' else energy_consumption,
            'starttijd datum': departure_time.strftime('%d/%m/%Y  %H:%M:%S'),  # Modified format
            'eindtijd datum': arrival_time.strftime('%d/%m/%Y  %H:%M:%S'),    # Modified format
            'omloop nummer': available_bus
        })

        # Update the bus SOC after assigning it a trip
        bus_soc[available_bus] -= energy_consumption

        # Check if the bus needs to charge
        if bus_soc[available_bus] < 0.1 * 300:  # 10% of battery capacity
            # Calculate the charging time required to reach 90% SOC
            charging_time = timedelta(hours=(0.9 * 300 - bus_soc[available_bus]) / 150)  # Assuming 150 kW charging rate
            charging_start = arrival_time
            charging_end = charging_start + charging_time

            # Record the charging schedule with formatted start and end date-times
            bus_charging[available_bus].append({
                'startlocatie': 'ehvgar',
                'eindlocatie': 'ehvgar',
                'starttijd': charging_start.time(),
                'eindtijd': charging_end.time(),
                'activiteit': 'opladen',
                'buslijn': 'n/a',
                'energieverbruik': -(0.9 * 300 - bus_soc[available_bus]),
                'starttijd datum': charging_start.strftime('%d/%m/%Y  %H:%M:%S'),  # Modified format
                'eindtijd datum': charging_end.strftime('%d/%m/%Y  %H:%M:%S'),    # Modified format
                'omloop nummer': available_bus
            })

            # Update the bus SOC after charging
            bus_soc[available_bus] = 0.9 * 300

        # Check if the bus needs to travel to the garage for the next trip
        if bus_last_location[available_bus] != 'ehvgar' and end_location != 'ehvgar':
            # Schedule a material trip from the last location to the garage
            garage_trip_distance = afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['afstand in meters'].values[0]
            garage_trip_energy = calculate_energy_consumption(garage_trip_distance)
            garage_trip_travel_time = timedelta(minutes=int(afstandsmatrix[(afstandsmatrix['startlocatie'] == bus_last_location[available_bus]) & (afstandsmatrix['eindlocatie'] == 'ehvgar')]['min reistijd in min'].values[0]))
            garage_trip_arrival = arrival_time + garage_trip_travel_time

            bus_schedules.append({
                'startlocatie': bus_last_location[available_bus],
                'eindlocatie': 'ehvgar',
                'starttijd': arrival_time.time(),
                'eindtijd': garage_trip_arrival.time(),
                'activiteit': 'materiaal rit',
                'buslijn': 'n/a',
                'energieverbruik': garage_trip_energy,
                'starttijd datum': arrival_time.strftime('%d/%m/%Y  %H:%M:%S'),  # Modified format
                'eindtijd datum': garage_trip_arrival.strftime('%d/%m/%Y  %H:%M:%S'),  # Modified format
                'omloop nummer': available_bus
            })

            # Update the bus SOC after the material trip
            bus_soc[available_bus] -= garage_trip_energy

        # Update the last location of the bus
        bus_last_location[available_bus] = end_location

    # Combine the bus schedules and charging schedules
    for bus, charging_trips in bus_charging.items():
        bus_schedules.extend(charging_trips)

    # Convert the bus_schedules list to a DataFrame for output
    schedule_df = pd.DataFrame(bus_schedules)
    return schedule_df

def calculate_energy_consumption(distance):
    """Calculate the energy consumption for a bus trip based on the distance."""
    return distance * 2.5 / 1000  # kWh per meter

def calculate_total_cost(schedule_df):
    """
    Calculate the total cost of a schedule, aiming to minimize energy usage
    and the number of unique omloop nummers.
    """
    total_energy_cost = schedule_df['energieverbruik'].abs().sum()  # Sum of absolute energy usage
    unique_omloop_cost = len(schedule_df['omloop nummer'].unique()) * 1000  # Penalize for more unique omloop nummers

    # Total cost balances between energy use and minimizing unique omloop nummers
    total_cost = total_energy_cost + unique_omloop_cost
    return total_cost

def generate_initial_solution(dienstregeling, afstandsmatrix):
    """Generate the initial feasible solution using the existing function."""
    return generate_feasible_plan(dienstregeling, afstandsmatrix)

def modify_schedule(schedule_df):
    """Make a small random modification to the schedule for Simulated Annealing."""
    modified_schedule = schedule_df.copy()
    
    # Try to merge trips into fewer omloop nummers by moving trips between omlopen
    unique_omloop_nummers = modified_schedule['omloop nummer'].unique()
    
    if len(unique_omloop_nummers) > 1:
        # Randomly choose two trips with different omloop nummers
        omloop_1, omloop_2 = random.sample(list(unique_omloop_nummers), 2)
        
        # Get two random indices corresponding to these omloop nummers
        idx1 = modified_schedule[modified_schedule['omloop nummer'] == omloop_1].sample(1).index[0]
        idx2 = modified_schedule[modified_schedule['omloop nummer'] == omloop_2].sample(1).index[0]
        
        # Attempt to reassign the omloop nummer of one trip to reduce total omlopen
        modified_schedule.at[idx1, 'omloop nummer'] = omloop_2
    
    # Ensure constraints like energy limits and SOC are respected (additional checks can be added here)

    return modified_schedule

def simulated_annealing(dienstregeling, afstandsmatrix, initial_temp=100000, cooling_rate=0.00003, max_iter=100000):
    current_solution = generate_feasible_plan(dienstregeling, afstandsmatrix)
    current_cost = calculate_total_cost(current_solution)
    
    best_solution = current_solution
    best_cost = current_cost
    
    temperature = initial_temp

    for i in range(max_iter):
        new_solution = modify_schedule(current_solution)
        new_cost = calculate_total_cost(new_solution)
        
        cost_diff = new_cost - current_cost
        if cost_diff < 0 or np.exp(-cost_diff / temperature) > random.random():
            current_solution = new_solution
            current_cost = new_cost
            
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost

        temperature *= (1 - cooling_rate)
        if temperature < 1e-3:
            break

    # Print the amount of unique omloop nummer in the optimized plan
    unique_omloop_count = len(best_solution['omloop nummer'].unique())
    print(f"Number of unique omloop nummer in optimized plan: {unique_omloop_count}")
    
    return best_solution


def save_plan_to_excel(schedule_df, output_file):
    """Save the consolidated bus planning schedule to an Excel file on a single sheet."""
    with pd.ExcelWriter(output_file) as writer:
        schedule_df.to_excel(writer, sheet_name='Consolidated Schedule', index=False)

# Example usage
input_file = 'Connexxion data - 2024-2025.xlsx'  # Input Excel file
output_file = 'optimized_feasible_plan_consolidated.xlsx'  # Output Excel file

# Load the data from the Excel file
dienstregeling = pd.read_excel(input_file, sheet_name='Dienstregeling')
afstandsmatrix = pd.read_excel(input_file, sheet_name='Afstandsmatrix')

# Run the Simulated Annealing optimization
optimized_plan_df = simulated_annealing(dienstregeling, afstandsmatrix)

# Save the optimized plan to an Excel file
save_plan_to_excel(optimized_plan_df, output_file)


Number of unique omloop nummer in optimized plan: 3
